In [1]:
import pandas as pd
import joblib
import numpy as np
from sklearn.impute import SimpleImputer
from tabulate import tabulate

#cicdataset = pd.read_csv('C:/Users/katsa/Documents/cicids2017.csv')
cicdataset = pd.read_csv("C:/Users/katsa/OneDrive/Jupyter_files/shallow_models_online/cic_test_sample_binary.csv")

In [2]:
cicdataset = cicdataset.sample(frac=0.1)
cicdataset.reset_index(drop=True, inplace=True)
cicdataset.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,119990,3,5,431,2267,431,0,143.666667,248.837966,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,53,31485,2,2,88,344,44,44,44.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,53,32349,2,2,60,288,30,30,30.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,443,5492489,9,6,367,3710,210,0,40.777778,75.805306,...,32,151163.0,0.0,151163,151163,5341322.0,0.0,5341322,5341322,BENIGN
4,53,184,2,2,84,204,42,42,42.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [3]:
# Define the discriminator model
import torch
from torch import nn


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(data_dim, 40),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(40, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, 16),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(16, 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, data):
        validity = self.model(data)
        return validity
    
    

In [4]:
# save all the different network configurations
import glob

discr_models_lst = glob.glob('./models/discr_model*.pth')
print(discr_models_lst)

['./models\\discr_model-lr=0.0002-batch=64-epochs=0.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=1.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=10.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=11.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=12.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=13.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=14.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=15.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=16.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=17.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=18.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=19.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=2.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=20.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=21.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=22.pth', './models\\discr_model-lr=0.0002-batch=64-epochs=23.pth', './models\\discr

In [5]:
discr_models_lst[0][9:-3]

'discr_model-lr=0.0002-batch=64-epochs=0.'

In [6]:
data_dim = 78
discriminator = Discriminator()

In [7]:
# selected_features = ['Destination Port', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
#                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
#                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 
#                      'Bwd Packet Length Std', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
#                      'Flow IAT Min', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Std',
#                      'Bwd IAT Max', 'Bwd IAT Min', 'Min Packet Length', 'Max Packet Length', 
#                      'Packet Length Mean', 'Packet Length Std', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 
#                      'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count']

# added_features = ['Fwd PSH Flags', 'Fwd URG Flags', 'Bwd PSH Flags', 'Bwd URG Flags']

# #extra_features = ['Active Max', 'Idle Max' ]

# computed_features = ['Flow Duration', 'Flow Bytes/s', 'Flow Packets/s', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Variance']

# selected_features_total = (selected_features + 
#                            added_features + 
#                            computed_features)
    

In [8]:
print(cicdataset.columns)

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

In [9]:
minmaxscaler = joblib.load('minmaxscaler')

In [10]:
simp = SimpleImputer(keep_empty_features=True)

cicdataset_labels = cicdataset['Label']

#cicdataset = cicdataset[selected_features_total]
cicdataset = cicdataset.iloc[:, :-1]

cicdataset.replace([np.inf, -np.inf], np.nan, inplace=True)

cicdataset_features = simp.fit_transform(cicdataset)

cicdataset_features = minmaxscaler.transform(cicdataset_features)


In [11]:
num_samples = len(cicdataset_features)
total_benign_conn = cicdataset_labels.value_counts().BENIGN
total_attack_conn = cicdataset_labels.value_counts().ATTACK
best_accuracy = -1
best_pair = {}

for PATH_discr in discr_models_lst:
    discriminator.load_state_dict(torch.load(PATH_discr))
    discriminator.eval()
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    for i in range(len(cicdataset_features)):
        row_label = cicdataset_labels[i]
        row_features = cicdataset_features[i]
        row_features = torch.tensor(row_features).float()
        model_output = discriminator(row_features)
        if model_output < 0.5:
            attack = True
        else:
            attack = False
        if (row_label == 'BENIGN' and attack == False):
            true_negative += 1
        if (row_label == 'ATTACK' and attack == True):
            true_positive += 1
        if (row_label == 'BENIGN' and attack == True):
            false_positive += 1
        if (row_label == 'ATTACK' and attack == False):
            false_negative += 1
    
    benign_recall = true_negative / (true_negative + false_positive)
    attack_recall = true_positive / (true_positive + false_negative)
    
    benign_precision = true_negative / (true_negative + false_negative)
    attack_precision = true_positive / (true_positive + false_positive)
    
    benign_f1 = 2 * (benign_recall * benign_precision) / (benign_recall + benign_precision)
    attack_f1 = 2 * (attack_recall * attack_precision) / (attack_recall + attack_precision)
    
    accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_pair  = {PATH_discr : accuracy}
    
    data = {
        "BENIGN": {"Precision": benign_precision, "Recall": benign_recall, "F1-Score": benign_f1},
        "ATTACK": {"Precision": attack_precision, "Recall": attack_recall, "F1-Score": attack_f1}
    }
    
    # Convert the dictionary to a list of lists
    table = [[row_name] + list(row_data.values()) for row_name, row_data in data.items()]
    # Get the column names
    #column_names = [""] + list(data.values()[0].keys())
    column_names = ["Precision", "Recall", "F1-Score"]
    # Print the table using tabulate
    #print(tabulate(table, headers=column_names, tablefmt="fancy_grid"))
    
    str1 = 'Results of ' + str(PATH_discr) + ' ...'
    str2 = "Overall Accuracy: " + str(accuracy)
    str3 = tabulate(table, headers=column_names, tablefmt="fancy_grid")
    output = str1 + "\n" + str2 + "\n" + str3 + "\n" + "###################################################"
    print(output)
    
    output2 = str2 + "\n" + str(data)
    file_name = './test_eval2/' + 'output-' + PATH_discr[9:-3] + 'txt'
    file = open(file_name, 'w')
    a = file.write(output2)
    file.close()

print("Best model overall is: " + str(best_pair))

Results of ./models\discr_model-lr=0.0002-batch=64-epochs=0.pth ...
Overall Accuracy: 0.9416398770622108
╒════════╤═════════════╤══════════╤════════════╕
│        │   Precision │   Recall │   F1-Score │
╞════════╪═════════════╪══════════╪════════════╡
│ BENIGN │    0.961967 │ 0.96552  │   0.96374  │
├────────┼─────────────┼──────────┼────────────┤
│ ATTACK │    0.857065 │ 0.844137 │   0.850552 │
╘════════╧═════════════╧══════════╧════════════╛
###################################################
Results of ./models\discr_model-lr=0.0002-batch=64-epochs=1.pth ...
Overall Accuracy: 0.9465856501925319
╒════════╤═════════════╤══════════╤════════════╕
│        │   Precision │   Recall │   F1-Score │
╞════════╪═════════════╪══════════╪════════════╡
│ BENIGN │    0.971605 │ 0.961606 │   0.96658  │
├────────┼─────────────┼──────────┼────────────┤
│ ATTACK │    0.849561 │ 0.885258 │   0.867042 │
╘════════╧═════════════╧══════════╧════════════╛
###################################################
